In [1]:
import matplotlib.pyplot as plt
import math
import pandas as pd
import numpy as np
import json
# from matplotlib.figure import Figure
%matplotlib widget

In [2]:
csv_df = pd.read_csv('HF_1215_full.csv')#change name


# Initializing arrays needed


In [3]:
# init arrays
reference_point =[]
time_arr = []
x_arr = []
y_arr = []
x_arr_latlong= []
y_arr_latlong= []
cov_x = []
cov_y = []
x_speed_cov = []
y_speed_cov = []
speed_y = []
speed_x = []
status_flags = []
gps_status = []
speed_tot = []
cov_net = []
counter_arr =[]
pos_net_xy=[]
roll_arr =[]
roll_degrees_arr=[]
pitch_arr = []
yaw_arr = []
heading_arr = []
activity_type = []

#warning flags
gq7_roll_pitch_warning = []
gq7_heading_warning = []
gq7_position_warning = []
gq7_velocity_warning = []
gq7_imu_bias_warning = []
gq7_gnss_clk_warning = []
gq7_antenna_lever_arm_warning = []
gq7_mounting_transform_warning = []
gq7_time_sync_warning = []
gq7_solution_error = []

#RTK
integer_fix = []
differential_corrections = []
tight_coupling = []
count_arr = 0
integer_fix_count = 0
differential_corrections_count = 0
dongle_signal_quality = []
dongle_sum =0
task_waypoints_array = []
task_waypoints_x_array = []
task_waypoints_y_array = []
task_waypoints_lat_array = []
task_waypoints_lon_array = []
high_res_path = []
high_res_path_x = []
high_res_path_y = []

# Populate Arrays

In [4]:
def populate_arrays(file):
    time_arr.clear()
    x_arr_latlong.clear()
    y_arr_latlong.clear()
    cov_x.clear()
    cov_y.clear()
    x_speed_cov.clear()
    y_speed_cov.clear()
    speed_y.clear()
    cov_net.clear()
    speed_x.clear()
    status_flags.clear()
    gps_status.clear()
    counter_arr.clear()
    speed_tot.clear()
    heading_arr.clear()
    activity_type.clear()
    
    
    for ind, entry in enumerate(file.values):
        if(entry[1] != 0 and entry[2] != 0):
            counter_arr.append(ind)
            time_arr.append(entry[0])
            x_arr_latlong.append(entry[1]/100)
            y_arr_latlong.append(entry[2]/100)
            cov_x.append(entry[9])
            cov_y.append(entry[10])
            cov_net.append(math.sqrt(entry[3]**2+entry[4]**2))
            gps_status.append(entry[11])
            status_flags.append(entry[12])
            speed_x.append(float(entry[14]))
            x_speed_cov.append(entry[15])
            speed_y.append(float(entry[16]))
            y_speed_cov.append(entry[17])
            speed_tot.append(math.sqrt(entry[14]**2 + entry[16]**2)) 
    
    
def populate_arrays_rtk(file):
    time_arr.clear()
    x_arr_latlong.clear()
    y_arr_latlong.clear()
    cov_x.clear()
    cov_y.clear()
    x_speed_cov.clear()
    y_speed_cov.clear()
    speed_y.clear()
    cov_net.clear()
    speed_x.clear()
    status_flags.clear()
    gps_status.clear()
    counter_arr.clear()
    speed_tot.clear()
    integer_fix.clear()
    tight_coupling.clear()
    differential_corrections.clear()
    pitch_arr.clear()
    yaw_arr.clear()
    integer_fix_count = 0
    differential_corrections_count = 0
    count_arr = 0
    dongle_sum =0
    
    for ind, entry in enumerate(file.values):
        if(entry[1] != 0 and entry[2] != 0):
            count_arr += 1
            counter_arr.append(ind)
            time_arr.append(entry[0])
            x_arr_latlong.append(entry[1])
            y_arr_latlong.append(entry[2])
            heading_arr.append(entry[3])
            cov_x.append(entry[10])
            cov_y.append(entry[11])
            cov_net.append(math.sqrt(entry[10]**2+entry[11]**2))
            gps_status.append(entry[13])
            status_flags.append(entry[14])
            speed_x.append(float(entry[17]))
            x_speed_cov.append(entry[18])
            speed_y.append(float(entry[19]))
            y_speed_cov.append(entry[20])
            speed_tot.append(math.sqrt(entry[15]**2 + entry[17]**2))
            integer_fix.append(entry[21] or entry[27])
            tight_coupling.append(entry[25])
            differential_corrections.append(entry[22])
            if(entry[22] ==1):
                differential_corrections_count += 1
            if(entry[21] or entry[27] == 1):
                integer_fix_count += 1
            roll,pitch,yaw = euler_from_quaternion(entry[3], entry[4], entry[5], entry[6])
            roll_arr.append(roll)
            pitch_arr.append(pitch)
            roll_degrees_arr.append(math.degrees(roll))
            yaw_arr.append(yaw)
            dongle_signal_quality.append(entry[38])
            dongle_sum += entry[38]
    
    print("Percentage of time spent with integer fix: " + str(integer_fix_count/count_arr)) 
    print("Percentage of time spent with differential corections: " + str(differential_corrections_count/count_arr)) 
    print("average donge signal quality: " + str(dongle_sum/count_arr))

def init_ref_point():
    reference_point.append(x_arr_latlong[0])
    reference_point.append(y_arr_latlong[0])

def generate_xy():
    x_arr.clear()
    y_arr.clear()
    pos_net_xy.clear()

    for i in range(len(x_arr_latlong)):
        x,y = ll2xy(reference_point[0], reference_point[1], x_arr_latlong[i], y_arr_latlong[i])
        x_arr.append(x)
        y_arr.append(y)
        pos_net_xy.append(math.sqrt(x**2 + y**2))
        
    for i in range(len(task_waypoints_array)):
        x,y = ll2xy(reference_point[0], reference_point[1], task_waypoints_array[i][0], task_waypoints_array[i][1])
        task_waypoints_x_array.append(x)
        task_waypoints_y_array.append(y)
        
def detwarningarrs():
    gq7_roll_pitch_warning.clear()
    gq7_heading_warning.clear()
    gq7_position_warning.clear()
    gq7_velocity_warning.clear()
    gq7_imu_bias_warning.clear()
    gq7_gnss_clk_warning.clear()
    gq7_antenna_lever_arm_warning.clear()
    gq7_mounting_transform_warning.clear()
    gq7_time_sync_warning.clear()
    gq7_solution_error.clear()
    count_antennae_warnings = 0
    count_arr = 0
    
    for j in status_flags:

        bin_num = int2bin(j)
        bin_array = list(bin_num)
#         print(bin_array)
        bin_array.reverse()
#         print(bin_array)
        for count, value in enumerate(bin_array):
            count_arr += 1
            if(count == 2):
                gq7_roll_pitch_warning.append(value)
            if(count == 3):
                gq7_heading_warning.append(value)
            if(count == 4):
                gq7_position_warning.append(value)
            if(count == 5):
                gq7_velocity_warning.append(value)
            if(count == 6):
                gq7_imu_bias_warning.append(value)
            if(count == 7):
                gq7_gnss_clk_warning.append(value)
            if(count == 8):
                gq7_antenna_lever_arm_warning.append(value)
                count_antennae_warnings +=1
            if(count == 9):
                gq7_mounting_transform_warning.append(value)
            if(count == 10):
                gq7_time_sync_warning.append(value)
            if(count > 11 and count < 16):
                gq7_solution_error.append(value)
                
    
def euler_from_quaternion(x, y, z, w):
        """
        Convert a quaternion into euler angles (roll, pitch, yaw)
        roll is rotation around x in radians (counterclockwise)
        pitch is rotation around y in radians (counterclockwise)
        yaw is rotation around z in radians (counterclockwise)
        """
        t0 = +2.0 * (w * x + y * z)
        t1 = +1.0 - 2.0 * (x * x + y * y)
        roll_x = math.atan2(t0, t1)
     
        t2 = +2.0 * (w * y - z * x)
        t2 = +1.0 if t2 > +1.0 else t2
        t2 = -1.0 if t2 < -1.0 else t2
        pitch_y = math.asin(t2)
     
        t3 = +2.0 * (w * z + x * y)
        t4 = +1.0 - 2.0 * (y * y + z * z)
        yaw_z = math.atan2(t3, t4)
        return roll_x, pitch_y, yaw_z # in radians
    
def parse_json_task(task_name):
        task_waypoints_array.clear()
        task_waypoints_x_array.clear()
        task_waypoints_y_array.clear()
        task_waypoints_lat_array.clear()
        task_waypoints_lon_array.clear()
        high_res_path_y.clear()
        high_res_path_x.clear()
        high_res_path.clear()
        with open('/app/tasks/' + task_name + '.json') as f:
            file = json.load(f)
        payload = file["payload"]
        data = payload["data"]
        vehicle_task_plan = data["vehicleTaskPlan"]
        route = vehicle_task_plan["route"]
        waypoints = route["waypoints"]
        previous_side = ""
        for i, wp in enumerate(waypoints):
            geometries = wp["geometries"]
            for j, geometry in enumerate(geometries):
                coordinates = geometry["coordinates"]
                for k, coordinate in enumerate(coordinates):
                    x = coordinate["lat"]
                    y = coordinate["lng"]
                    task_waypoints_lat_array.append(x)
                    task_waypoints_lon_array.append(y)
                    point = [x,y]
                    task_waypoints_array.append(point)     
                    
def get_stamped_pose_array(stamped_pose1, stamped_pose2, points_per_meter):
        """
        PARAMETERS
        Generate an array of StampedPose objects with evenly distributed stamped_poses between stamped_pose1 and stamped_pose2, including stamped_pose1 but not stamped_pose2
        param pose1:                Pose() initial pose
        param pose2:                Pose() pose not included
        params points_per_meter:    wp/m
        """
        x1, y1 = stamped_pose1[0], stamped_pose1[1]
        x2, y2 = stamped_pose2[0], stamped_pose2[1]
        L = np.hypot(np.abs(x2-x1), np.abs(y2-y1))
        intermediate_pts = int((points_per_meter*L)-2)
        pts_np = np.linspace([stamped_pose1[0], stamped_pose1[1]],[stamped_pose2[0], stamped_pose2[1]], num=intermediate_pts+2,endpoint=False)
        for i in pts_np:
            high_res_path_x.append(i[0])
            high_res_path_y.append(i[1])
#          
#             for j in i:
                
#                 
                        
        
        
#         high_res_path_x.append(pts_np[0][0])
#         high_res_path_x.append(pts_np[0][1])


# Plot Functions

In [5]:
def plotpath(x,y,status,flag,num):
    x_status = []
    y_status = []
    x_flag = []
    y_flag = []
    x_gq7_antenna_lever_arm_warning = []
    y_gq7_antenna_lever_arm_warning = []
    for i in range(0,len(status)):
        if (integer_fix[i] != 1):
            x_status.append(x[i])
            y_status.append(y[i])
        if (gq7_position_warning[i] == 1):
            x_flag.append(x[i])
            y_flag.append(y[i])
       
    plt.plot(x,y,'.-')
    plt.plot(high_res_path_x, high_res_path_y, 'or')#,markersize=15)
#     plt.plot(x_flag, y_flag,'xg')#,markersize=10)
#     plt.plot(x_status, y_status,'xr')#,markersize=10)
    plt.xlabel('x [m]')
    plt.ylabel('y [m]')
    plt.legend(('localization 1','Task Route Points','not integer fix', 'localization 2'))
    
    
def plotpathclean():
    plt.plot(x_arr_latlong, y_arr_latlong,'.-')
    plt.plot(high_res_path_x, high_res_path_y, 'or')#,markersize=15)
    plt.xlabel('lat')
    plt.ylabel('lon')
    plt.legend(('localization 1','Task Route Points'))
    
def create_plots_vel_covel():
    fix, axis = plt.subplots(2,2)
    axis[0][0].plot(speed_x)
    axis[1][0].plot(x_speed_cov)
    axis[0][1].plot(speed_y)
    axis[1][1].plot(y_speed_cov)
    
    axis[0][0].set(xlabel="x velocity", ylabel="m/s")
    axis[1][0].set(xlabel="x velocity covariance", ylabel="")
    axis[0][1].set(xlabel="y velocity", ylabel="m/s")
    axis[1][1].set(xlabel="y velocity covariance", ylabel="")

def plotspeed():
    plt.plot(speed_tot)
    plt.xlabel('points')
    plt.ylabel('m/s') 
    plt.title('speed')
    
def create_plots_pos_cov():
    fix, axis = plt.subplots(3,2)
    
    axis[0][0].plot(x_arr_latlong)
    axis[1][0].plot(x_arr)
    axis[2][0].plot(cov_x)
    axis[0][1].plot(y_arr_latlong)
    axis[1][1].plot(y_arr)
    axis[2][1].plot(cov_y)
    
    axis[0][0].set(xlabel="lat", ylabel="")
    axis[1][0].set(xlabel="x", ylabel="m")
    axis[2][0].set(xlabel="covariance x", ylabel="")
    axis[0][1].set(xlabel="lon", ylabel="m/s")
    axis[1][1].set(xlabel="y", ylabel="m/s")
    axis[2][1].set(xlabel="covariance y", ylabel="")
    
def create_plots_vel_velwarning():
    fix, axis = plt.subplots(2,1)
    axis[0].plot(speed_tot)
    axis[1].plot(gq7_velocity_warning)
    
    axis[0].set(xlabel="velocity", ylabel="m/s")
    axis[1].set(xlabel="velocity warning", ylabel="")
    
def ll2xy(ref_lat, ref_lon, lat, lon):
        const_Re = 6371000 
        x = (float(lat) - float(ref_lat)) * (math.pi / 180) * const_Re
        y = (float(lon) - float(ref_lon)) * (math.pi / 180) * const_Re * math.cos(float(lat) * math.pi / 180)
        
        return x,y
    
def int2bin(number):
    return "{0:015b}".format(int(number))

                
def plotcovandwarnings():
    fix, axis = plt.subplots(3,1)
    axis[0].plot(cov_net)
    axis[1].plot(gq7_velocity_warning)
    axis[2].plot(gq7_position_warning)
    
    axis[0].set(xlabel="cov_net", ylabel="")
    axis[1].set(xlabel="velocity warning", ylabel="")
    axis[2].set(xlabel="pos warning", ylabel="")
    
def plotallwarnings():
    fix, axis = plt.subplots(10,1,figsize=(15,15))
    plt.ylim([0,1])
    
    axis[0].plot(gq7_roll_pitch_warning)
    axis[1].plot(gq7_heading_warning)
    axis[2].plot(gq7_position_warning)
    axis[3].plot(gq7_velocity_warning)
    axis[4].plot(gq7_imu_bias_warning)
    axis[5].plot(gq7_gnss_clk_warning)
    axis[6].plot(gq7_antenna_lever_arm_warning)
    axis[7].plot(gq7_mounting_transform_warning)
    axis[8].plot(gq7_time_sync_warning)
    axis[9].plot(gq7_solution_error)
    
    axis[0].set(xlabel="gq7_roll_pitch_warning", ylabel="gq7_roll_pitch_warning")
    axis[1].set(xlabel="gq7_heading_warning", ylabel="gq7_heading_warning")
    axis[2].set(xlabel="gq7_position_warning", ylabel="gq7_position_warning")
    axis[3].set(xlabel="gq7_velocity_warning", ylabel="gq7_velocity_warning")
    axis[4].set(xlabel="gq7_imu_bias_warning", ylabel="gq7_imu_bias_warning")
    axis[5].set(xlabel="gq7_gnss_clk_warning", ylabel="gq7_gnss_clk_warning")
    axis[6].set(xlabel="gq7_antenna_lever_arm_warning", ylabel="gq7_antenna_lever_arm_warning")
    axis[7].set(xlabel="gq7_mounting_transform_warning", ylabel="gq7_mounting_transform_warning")
    axis[8].set(xlabel="gq7_time_sync_warning", ylabel="gq7_time_sync_warning")
    axis[9].set(xlabel="gq7_solution_error", ylabel="gq7_solution_error")
    
def dist(x1,y1,x2,y2):
    return math.sqrt((x2-x1)**2+(y2-y1)**2)

def sendFileToPopulateAndDetWarnings(name):
    if len(name.columns)>20:
        populate_arrays_rtk(name)
    else:
        populate_arrays(name)

    detwarningarrs()  #Populate warning flags

# updated firmware and drivers

In [6]:
sendFileToPopulateAndDetWarnings(csv_df)
parse_json_task("current_task_BIN_multipoints_full")
init_ref_point()
generate_xy()

for i in range(len(task_waypoints_x_array)-1):
    p1 = [task_waypoints_x_array[i], task_waypoints_y_array[i]]
    p2 = [task_waypoints_x_array[i+1], task_waypoints_y_array[i+1]]
    get_stamped_pose_array(p1, p2, 4)


# print(len(high_res_path_y))
# print(len(task_waypoints_lat_array))

Percentage of time spent with integer fix: 1.0
Percentage of time spent with differential corections: 1.0
average donge signal quality: 0.0


# Speed

In [7]:
create_plots_vel_covel()

plt.figure()
plotspeed()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Position and cov

In [8]:
create_plots_pos_cov()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
plotallwarnings()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# warning plots

In [10]:
create_plots_vel_velwarning()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
plotcovandwarnings()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
plt.figure()
plt.plot(gps_status)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
plt.figure()
plt.plot(status_flags)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# path plot

In [14]:
# ind = slice(5350, len(x_arr_latlong)-0) #T101--B22-2021-08-18
#ind = slice(6850, len(x_arr_latlong)-800) #T101--B22-2021-08-30
#ind = slice(6000, len(x_arr_latlong)-2600) #13-10-21
plt.figure()
# ind1 = slice(len(x_arr_latlong))
ind2 = slice(len(x_arr_latlong))
x_short, y_short, status_short, flag_short, counter_arr_short = x_arr[ind2], y_arr[ind2], gps_status[ind2], status_flags[ind2], counter_arr[ind2]
arrx = np.array(counter_arr_short)
plotpath(x_short,y_short,status_short,flag_short, 1)



# m, b = np.polyfit(x_short, y_short, 1)
# print(m)
# print(b)
#compare = pd.read_csv('13-10-21/TwinOaksCS_S-6.csv')#change name
#sendFileToPopulateAndDetWarnings(compare)
#generate_xy()
#x_short, y_short, status_short, flag_short = x_arr[ind1], y_arr[ind1], gps_status[ind1], status_flags[ind1]
#plotpath(x_short,y_short,status_short,flag_short, 2)
data = np.linspace(-2.3,0)
#plt.plot(data, data*m + b)

plt.grid(which='both')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
plt.figure()
plotpathclean()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
plt.figure()
plt.plot(counter_arr, dongle_signal_quality)
plt.title('dongal signal quality over time  - 19/10/21 USA')
plt.xlabel('points')
plt.ylabel('quality')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'quality')

In [17]:
plt.figure()
plt.plot(counter_arr, integer_fix)
plt.title('integer fix over time - 19/10/21 USA')
plt.xlabel('points')
plt.ylabel('integer fix')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'integer fix')

In [18]:
plt.figure()
ind1 = slice(1,len(heading_arr))
heading_arr = heading_arr[ind1]
plt.plot(heading_arr)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
import base64
import pandas as pd
from IPython.display import HTML

x_arr_latlong_new = []
y_arr_latlong_new = []

for i in range(len(x_arr_latlong)):
    if not i % 10:
        x_arr_latlong_new.append(x_arr_latlong[i])
        y_arr_latlong_new.append(y_arr_latlong[i])

print(x_arr_latlong_new)

[32.509379342, 32.509379305, 32.509379264, 32.509379136, 32.509379107, 32.509379142, 32.50937917, 32.509379097, 32.509377924, 32.509376841, 32.509375855, 32.509375241, 32.509373938, 32.509373033, 32.509372246, 32.509371478, 32.509370436, 32.509369933, 32.509368767, 32.509368358, 32.509367454, 32.509366706, 32.509366551, 32.509365153, 32.509364536, 32.509363876, 32.50936325, 32.509362665, 32.509361253, 32.509361396, 32.509360731, 32.509359518, 32.509358625, 32.509357316, 32.509356951, 32.509356146, 32.509355242, 32.509354516, 32.509354145, 32.509352745, 32.50935224, 32.509351621, 32.509350646, 32.509349951, 32.5093498, 32.509349438, 32.509348612, 32.50934856, 32.509347939, 32.509347294, 32.509346753, 32.509346419, 32.509346108, 32.509345112, 32.509344544, 32.50934416, 32.509343283, 32.509342344, 32.509341694, 32.509341074, 32.509340777, 32.509340248, 32.50933955, 32.509339258, 32.509338858, 32.509336927, 32.50933632, 32.509335008, 32.509334285, 32.509333157, 32.509331966, 32.509331538, 

In [20]:
csv_df.iloc[::10, :][['Latitude','Longitude']].apply(lambda x: x/100).to_csv("~/yanir.csv") #possibly the coolest thing ive ever seen

In [21]:
csv_df.iloc[::, :][['Latitude','Longitude']].to_csv("~/NIR.csv") #possibly the coolest thing ive ever seen